In [36]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

### 1st layer order

In [144]:
gid = 0
srv = 8
c = 0
k = 0
dataset = 1
test_error = {}
for mode in [3, 10, 14]:
    test_error[mode] = {}
    for fold_num in range(5):
        test_error[mode][fold_num] = {}
        for lr in [0.001, 0.01]:
            test_error[mode][fold_num][lr] = {}
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'], 1)):
                f_name = '{}_7'.format(mode)
                o = "\', \'".join(str(x) for x in order)
                directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/20000/0.0/".format(f_name, dataset, fold_num, lr)
                filename = "valid-error-[\'{}\'].npy".format(o)
                full_path = directory + filename
                my_file = Path(full_path)
                e = np.load(full_path).item()
                test_error[mode][fold_num][lr][order] = {}
                for iters in range(1000, 20001, 1000):
                    test_error[mode][fold_num][lr][order][iters] = e[iters]

In [4]:
appliance_contri = {'hvac':0.83003428, 'fridge':0.0827564, 'dr':0.06381463, 'dw':0.01472098, 'mw':0.00867371}

In [145]:
error = {}
for mode in [3, 10, 14]:
    error[mode] = {}
    for fold_num in range(5):
        error[mode][fold_num] = {}
        for lr in [0.001 ,0.01]:
            error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                error[mode][fold_num][lr][order] = {}
                for iters in range(1000, 20001, 1000):
                    apps = test_error[mode][fold_num][lr][order][iters].keys()
                    error[mode][fold_num][lr][order][iters] = 0
                    for k in apps:
                        error[mode][fold_num][lr][order][iters] += test_error[mode][fold_num][lr][order][iters][k]/appliance_contri[k]

In [146]:
b_error = {}
for mode in [3, 10, 14]:
    b_error[mode] = {}
    for fold_num in range(5):
        b_error[mode][fold_num] = {}
        for lr in [0.001 ,0.01]:
            b_error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
    #             b_error[fold_num][lr][order] = {}
    #             for iters in range(1000, 100001, 1000):
                b_error[mode][fold_num][lr][order] = pd.Series(error[mode][fold_num][lr][order]).min()

In [148]:
mean = {}
for mode in [3, 10, 14]:
    mean[mode] = 0

    for fold in range(5):
        mean[mode] += pd.DataFrame(b_error[mode][fold])

mean[3]

,0.001,0.01
dr,4283.462674,4186.296805
dw,3793.215680,3793.093222
fridge,1664.775863,1686.231292
hvac,972.115075,953.483620
mw,6632.347980,6629.968591


In [161]:
k = {}
for i in range(5):
    k[i] = pd.DataFrame(test_error[3][i][0.01][('mw',)]).min(axis=1).mean()

In [162]:
pd.Series(k).min()

5.960430382770709

### 2nd layer order

In [8]:
gid = 0
srv = 8
c = 0
k = 0
dataset = 1
test_error = {}
for mode in [3, 7, 8, 12]:
    test_error[mode] = {}
    for fold_num in range(5):
        test_error[mode][fold_num] = {}
        for lr in [0.001, 0.01, 0.1, 1.0]:
            test_error[mode][fold_num][lr] = {}
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'], 2)):
                if order[0] == 'dr' or order[0] == 'mw':
                    continue
                f_name = '7_{}'.format(mode)
                o = "\', \'".join(str(x) for x in order)
                directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/20000/0.0/".format(f_name, dataset, fold_num, lr)
                filename = "valid-error-[\'{}\'].npy".format(o)
                full_path = directory + filename
                my_file = Path(full_path)
                e = np.load(full_path).item()
                test_error[mode][fold_num][lr][order] = {}
                for iters in range(1000, 20001, 1000):
                    test_error[mode][fold_num][lr][order][iters] = e[iters]

In [9]:
error = {}
for mode in [3, 7, 8, 12]:
    error[mode] = {}
    for fold_num in range(5):
        error[mode][fold_num] = {}
        for lr in [0.001 ,0.01, 0.1, 1.0]:
            error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                error[mode][fold_num][lr][order] = {}
                for iters in range(1000, 20001, 1000):
                    apps = test_error[mode][fold_num][lr][order][iters].keys()
                    error[mode][fold_num][lr][order][iters] = 0
                    for k in apps:
                        error[mode][fold_num][lr][order][iters] += test_error[mode][fold_num][lr][order][iters][k]/appliance_contri[k]

In [10]:
b_error = {}
for mode in [3, 7, 8, 12]:
    b_error[mode] = {}
    for fold_num in range(5):
        b_error[mode][fold_num] = {}
        for lr in [0.001 ,0.01, 0.1, 1.0]:
            b_error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                b_error[mode][fold_num][lr][order] = pd.Series(error[mode][fold_num][lr][order]).min()

In [37]:
mean = {}
for mode in [3, 7, 8, 12]:
    mean[mode] = 0

    for fold in range(5):
        mean[mode] += pd.DataFrame(b_error[mode][fold])

In [38]:
mean[12][mean[12] < 4800]

0.001        0.010        0.100        1.000
dw     dr              NaN          NaN          NaN          NaN
       fridge          NaN          NaN          NaN          NaN
       hvac            NaN          NaN          NaN  4760.685832
       mw              NaN          NaN          NaN          NaN
fridge dr              NaN          NaN          NaN          NaN
       dw              NaN          NaN          NaN          NaN
       hvac    3155.002481  3114.944709  2953.497091  2733.476585
       mw              NaN          NaN          NaN          NaN
hvac   dr              NaN          NaN          NaN          NaN
       dw              NaN          NaN          NaN          NaN
       fridge  3203.310046  3091.940800  2994.143292  2790.366056
       mw              NaN          NaN          NaN          NaN

### 3rd layer

In [85]:
gid = 0
srv = 8
c = 0
k = 0
dataset = 1
test_error = {}
for mode in [3, 10, 14]:
    test_error[mode] = {}
    for fold_num in range(5):
        test_error[mode][fold_num] = {}
        for lr in [0.001, 0.01, 0.1, 1.0]:
            test_error[mode][fold_num][lr] = {}
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'], 3)):
                if (order[0] == 'fridge' and order[1] == 'hvac') or (order[0] == 'dw' and order[1] == 'hvac'):
#                     continue
                    f_name = '{}_7'.format(mode)
                    o = "\', \'".join(str(x) for x in order)
                    directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/20000/0.0/".format(f_name, dataset, fold_num, lr)
                    filename = "valid-error-[\'{}\'].npy".format(o)
                    full_path = directory + filename
                    my_file = Path(full_path)
                    e = np.load(full_path).item()
                    test_error[mode][fold_num][lr][order] = {}
                    for iters in range(1000, 20001, 1000):
                        test_error[mode][fold_num][lr][order][iters] = e[iters]

In [86]:
error = {}
for mode in [3, 10, 14]:
    error[mode] = {}
    for fold_num in range(5):
        error[mode][fold_num] = {}
        for lr in [0.001 ,0.01, 0.1, 1.0]:
            error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                error[mode][fold_num][lr][order] = {}
                for iters in range(1000, 20001, 1000):
                    apps = test_error[mode][fold_num][lr][order][iters].keys()
                    error[mode][fold_num][lr][order][iters] = 0
                    for k in apps:
                        error[mode][fold_num][lr][order][iters] += test_error[mode][fold_num][lr][order][iters][k]/appliance_contri[k]

In [87]:
b_error = {}
for mode in [3, 10, 14]:
    b_error[mode] = {}
    for fold_num in range(5):
        b_error[mode][fold_num] = {}
        for lr in [0.001 ,0.01, 0.1, 1.0]:
            b_error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                b_error[mode][fold_num][lr][order] = pd.Series(error[mode][fold_num][lr][order]).min()

In [88]:
mean = {}
for mode in [3, 10, 14]:
    mean[mode] = 0

    for fold in range(5):
        mean[mode] += pd.DataFrame(b_error[mode][fold])

In [93]:
mean[14][mean[14] < 8000]

0.001        0.010        0.100        1.000
dw     hvac dr              NaN          NaN          NaN          NaN
            fridge  7129.780395  6968.286781  7000.554143  6708.814588
            mw              NaN          NaN          NaN          NaN
fridge hvac dr              NaN          NaN          NaN  7912.300535
            dw      7004.734827  6908.935464  6711.936799  6631.146462
            mw              NaN          NaN          NaN          NaN

### 4th layer1

In [95]:
gid = 0
srv = 8
c = 0
k = 0
dataset = 1
test_error = {}
for mode in [3, 10, 14]:
    test_error[mode] = {}
    for fold_num in range(5):
        test_error[mode][fold_num] = {}
        for lr in [0.001, 0.01, 0.1, 1.0]:
            test_error[mode][fold_num][lr] = {}
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'], 4)):
                if (order[0] == 'fridge' and order[1] == 'hvac' and order[2] == 'dr') or (order[0] == 'fridge' and order[1] == 'hvac' and order[2] == 'dw') or (order[0] == 'dw' and order[1] == 'hvac' and order[2] == 'fridge'):
                    
#                     continue
                    f_name = '{}_7'.format(mode)
                    o = "\', \'".join(str(x) for x in order)
                    directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/20000/0.0/".format(f_name, dataset, fold_num, lr)
                    filename = "valid-error-[\'{}\'].npy".format(o)
                    full_path = directory + filename
                    my_file = Path(full_path)
                    e = np.load(full_path).item()
                    test_error[mode][fold_num][lr][order] = {}
                    for iters in range(1000, 20001, 1000):
                        test_error[mode][fold_num][lr][order][iters] = e[iters]

In [96]:
error = {}
for mode in [3, 10, 14]:
    error[mode] = {}
    for fold_num in range(5):
        error[mode][fold_num] = {}
        for lr in [0.001 ,0.01, 0.1, 1.0]:
            error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                error[mode][fold_num][lr][order] = {}
                for iters in range(1000, 20001, 1000):
                    apps = test_error[mode][fold_num][lr][order][iters].keys()
                    error[mode][fold_num][lr][order][iters] = 0
                    for k in apps:
                        error[mode][fold_num][lr][order][iters] += test_error[mode][fold_num][lr][order][iters][k]/appliance_contri[k]

In [97]:
b_error = {}
for mode in [3, 10, 14]:
    b_error[mode] = {}
    for fold_num in range(5):
        b_error[mode][fold_num] = {}
        for lr in [0.001 ,0.01, 0.1, 1.0]:
            b_error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                b_error[mode][fold_num][lr][order] = pd.Series(error[mode][fold_num][lr][order]).min()

In [98]:
mean = {}
for mode in [3, 10, 14]:
    mean[mode] = 0

    for fold in range(5):
        mean[mode] += pd.DataFrame(b_error[mode][fold])

In [102]:
mean[14][mean[14] < 13000]

0.001  0.010         0.100         1.000
dw     hvac fridge dr  12581.281925    NaN  12547.910689  12039.001233
                   mw           NaN    NaN           NaN           NaN
fridge hvac dr     dw  12615.495601    NaN  12998.490154  12067.729948
                   mw           NaN    NaN           NaN           NaN
            dw     dr  12551.618646    NaN  12680.074469  11917.862976
                   mw           NaN    NaN           NaN           NaN

In [105]:
gid = 0
srv = 8
c = 0
k = 0
dataset = 1
test_error = {}
for mode in [3, 10, 14]:
    test_error[mode] = {}
    for fold_num in range(5):
        test_error[mode][fold_num] = {}
        for lr in [0.001, 0.01, 0.1, 1.0]:
            test_error[mode][fold_num][lr] = {}
            
            for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw'], 5)):
                if (order[0] == 'dw' and order[1] == 'hvac' and order[2] == 'fridge' and order[3] == 'dr') or (order[0] == 'fridge' and order[1] == 'hvac' and order[2] == 'dw' and order[3] == 'dr') or (order[0] == 'fridge' and order[1] == 'hvac' and order[2] == 'dr' and order[3] == 'dw'):

                    
#                     continue
                    f_name = '{}_7'.format(mode)
                    o = "\', \'".join(str(x) for x in order)
                    directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/5000/0.0/".format(f_name, dataset, fold_num, lr)
                    filename = "valid-error-[\'{}\'].npy".format(o)
                    full_path = directory + filename
                    my_file = Path(full_path)
                    e = np.load(full_path).item()
                    test_error[mode][fold_num][lr][order] = {}
                    for iters in range(1000, 5001, 1000):
                        test_error[mode][fold_num][lr][order][iters] = e[iters]

In [106]:
error = {}
for mode in [3, 10, 14]:
    error[mode] = {}
    for fold_num in range(5):
        error[mode][fold_num] = {}
        for lr in [0.001 ,0.01, 0.1, 1.0]:
            error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                error[mode][fold_num][lr][order] = {}
                for iters in range(1000, 5001, 1000):
                    apps = test_error[mode][fold_num][lr][order][iters].keys()
                    error[mode][fold_num][lr][order][iters] = 0
                    for k in apps:
                        error[mode][fold_num][lr][order][iters] += test_error[mode][fold_num][lr][order][iters][k]/appliance_contri[k]

In [107]:
b_error = {}
for mode in [3, 10, 14]:
    b_error[mode] = {}
    for fold_num in range(5):
        b_error[mode][fold_num] = {}
        for lr in [0.001 ,0.01, 0.1, 1.0]:
            b_error[mode][fold_num][lr] = {}
            for order in test_error[mode][fold_num][lr].keys():
                b_error[mode][fold_num][lr][order] = pd.Series(error[mode][fold_num][lr][order]).min()

In [113]:
mean = {}
for mode in [3, 10, 14]:
    mean[mode] = 0

    for fold in range(5):
        mean[mode] += pd.DataFrame(b_error[mode][fold])

In [140]:
mean[14]

0.001         0.010         0.100  \
dw     hvac fridge dr mw  19830.638923  21051.438581  18997.515880   
fridge hvac dr     dw mw  19772.764651  20372.383656  19700.720492   
            dw     dr mw  19523.941693  20634.464784  18912.952320   

                                 1.000  
dw     hvac fridge dr mw  18977.701112  
fridge hvac dr     dw mw  18889.790791  
            dw     dr mw  18908.620774

In [141]:
k = {}
for i in range(5):
    k[i] = pd.DataFrame(test_error[14][i][1.0][('fridge', 'hvac', 'dr', 'dw', 'mw')]).min(axis=1).mean()

In [142]:
pd.Series(k).mean()

58.7451622960607

In [62]:
cand = {}
for mode in [3, 7, 8, 12]:
    for fold_num in range(5):
        c = pd.DataFrame(b_error[fold_num]).T.min().nsmallest(3).keys()
        cand[fold_num] = []
        for o in c:
            k = [x for x in APPLIANCE_ORDER[1:-1] if x not in o]
            a = o + (k[0],)
            cand[fold_num].append(a)

KeyError: 0

In [58]:
mean[3]

,,,0.001,0.01,0.1,1.0
dw,hvac,fridge,6790.599289,6683.820190,6641.279107,6532.560267
fridge,hvac,dw,6433.325371,6438.447413,6388.461021,6477.635545
hvac,fridge,dw,6811.548868,6681.094210,6688.810983,6557.817271


In [222]:
cand[4]

[('dw', 'fridge', 'hvac', 'dr'),
 ('fridge', 'hvac', 'dw', 'dr'),
 ('hvac', 'fridge', 'dw', 'dr')]

In [213]:
gid = 0
srv = 8
c = 0
k = 0
dataset = 3
test_error = {}
for fold_num in range(5):
    test_error[fold_num] = {}
    for lr in [0.01, 0.001, 0.1]:
        test_error[fold_num][lr] = {}
        for iters in [20000]:
            for order in cand[fold_num]:
                for p in [0.0]:
                    
                    try:
                        o = "\', \'".join(str(x) for x in order)
                        
                        directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/{}/".format(dataset, fold_num, lr, iters, p)
                        filename = "test-error-[\'{}\'].npy".format(o)
#                         print(filename)
                        full_path = directory + filename
                        my_file = Path(full_path)
#                         print(full_path)
                        e = np.load(full_path).item()
                        test_error[fold_num][lr][order] = {}
                        for iters in range(1000, 20001, 1000):
                            test_error[fold_num][lr][order][iters] = e[iters]
                    except:
                        o = " ".join(str(x) for x in order)
                        line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 3 {} {} {} {} {} &".format(gid, lr, 20000, p, fold_num, o))
                        print(line)
                        gid += 1
                        if gid == 4:
                            gid = 0

CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 3 0.01 20000 0.0 0 dw hvac fridge dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 3 0.01 20000 0.0 0 fridge hvac dw dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 3 0.01 20000 0.0 0 hvac fridge dw dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 3 0.001 20000 0.0 0 dw hvac fridge dr &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 3 0.001 20000 0.0 0 fridge hvac dw dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 3 0.001 20000 0.0 0 hvac fridge dw dr &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 3 0.1 20000 0.0 0 dw hvac fridge dr &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 3 0.1 20000 0.0 0 fridge hvac dw dr &
CUDA_VISIBLE_DEVICES=0 python cnn-tree.py 3 0.1 20000 0.0 0 hvac fridge dw dr &
CUDA_VISIBLE_DEVICES=1 python cnn-tree.py 3 0.01 20000 0.0 1 dr hvac fridge dw &
CUDA_VISIBLE_DEVICES=2 python cnn-tree.py 3 0.001 20000 0.0 1 dr hvac fridge dw &
CUDA_VISIBLE_DEVICES=3 python cnn-tree.py 3 0.1 20000 0.0 1 dr hvac fridge dw &
CUDA_VISIBLE_DEVICES=0 pytho

In [214]:
error = {}

for fold_num in range(5):
    error[fold_num] = {}
    for lr in [0.01]:
        error[fold_num][lr] = {}
        for order in test_error[fold_num][lr].keys():
            error[fold_num][lr][order] = {}
            for iters in range(1000, 20001, 1000):
                apps = test_error[fold_num][lr][order][iters].keys()
                error[fold_num][lr][order][iters] = 0
                for k in apps:
                    error[fold_num][lr][order][iters] += test_error[fold_num][lr][order][iters][k]/appliance_contri[k]

In [215]:
b_error = {}
iters = {}
for fold_num in range(5):
    b_error[fold_num] = {}
    iters[fold_num] = {}
    for lr in [0.01]:
        b_error[fold_num][lr] = {}
        iters[fold_num][lr] = {}
        for order in test_error[fold_num][lr].keys():
#             b_error[fold_num][lr][order] = {}
#             for iters in range(1000, 100001, 1000):
            b_error[fold_num][lr][order] = pd.Series(error[fold_num][lr][order]).min()
            iters[fold_num][lr][order] = pd.Series(error[fold_num][lr][order]).argmin()
        

/zf14/yj9xs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  del sys.path[0]


In [216]:
pd.DataFrame(b_error[2])

,,,,0.01
mw,fridge,hvac,dr,2730.886539


In [217]:
pd.DataFrame(iters[0])

,0.01


In [225]:
cnn_tree_params = {}
cnn_tree_params[1] = np.load("../code/baseline/result/cnn-tree-new-param-1.npy").item()
cnn_tree_params[3] = np.load("../code/baseline/result/cnn-tree-final-param-3.npy").item()

In [232]:
# get prediction for cnn tree
cnn_tree_error= {}
for dataset in [1, 3]:
    cnn_tree_error[dataset] = {}
    for fold_num in range(5):
#         cnn_tree_error[dataset][fold_num] = {}

        lr = cnn_tree_params[dataset][fold_num]['lr']
        iters = cnn_tree_params[dataset][fold_num]['iters']
        order = cnn_tree_params[dataset][fold_num]['order']

        o = "\', \'".join(str(x) for x in order)
        directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
        filename = "test-error-[\'{}\'].npy".format(o)

        full_path = directory + filename
        cnn_tree_error[dataset][fold_num] = np.load(full_path).item()[iters]

#         for idx, appliance in enumerate(order):
#             cnn_tree_error[dataset][fold_num][appliance] = cnn_pred[idx]
    

In [240]:
error = {}
error[1] = {}
error[1][0] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['dr', 'hvac', 'fridge', 'mw', 'dw'].npy").item()
error[1][1] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['hvac', 'dr', 'fridge', 'dw', 'mw'].npy").item()
error[1][2] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['fridge', 'hvac', 'mw', 'dw', 'dr'].npy").item()
error[1][3] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['hvac', 'mw', 'fridge', 'dw', 'dr'].npy").item()
error[1][4] = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['fridge', 'hvac', 'dw', 'mw', 'dr'].npy").item()

In [249]:
k = pd.DataFrame(error[1][0]).mean().min()*14 + pd.DataFrame(error[1][1]).mean().min()*14 + pd.DataFrame(error[1][2]).mean().min()*14 + pd.DataFrame(error[1][3]).mean().min()*13 + pd.DataFrame(error[1][4]).mean().min()*13

In [250]:
k/68

62.03123642656481

In [257]:
error = {}
error[1] = {}
error[1][0] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['dw', 'hvac', 'fridge', 'dr', 'mw'].npy").item()
error[1][1] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['fridge', 'hvac', 'dr', 'dw', 'mw'].npy").item()
error[1][2] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['mw', 'fridge', 'hvac', 'dr', 'dw'].npy").item()
error[1][3] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['hvac', 'fridge', 'mw', 'dr', 'dw'].npy").item()
error[1][4] = np.load("../code/baseline/cnn-tree/3/0/0.01/20000/0.0/test-error-['fridge', 'hvac', 'dw', 'dr', 'mw'].npy").item()

In [258]:
k = pd.DataFrame(error[1][0]).mean().min()*14 + pd.DataFrame(error[1][1]).mean().min()*14 + pd.DataFrame(error[1][2]).mean().min()*14 + pd.DataFrame(error[1][3]).mean().min()*13 + pd.DataFrame(error[1][4]).mean().min()*13

In [264]:
pd.DataFrame(error[1][1]).mean().min()

77.416244615561567